<a href="https://colab.research.google.com/github/simranjita/DBAF/blob/main/Classification_GRU_LSTM_Edge_Computing_Undersampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import sqrt
from numpy import concatenate

#from matplotlib a pyplot
from pandas import read_csv
from numpy import savetxt
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
from pandas import read_csv
from datetime import datetime

 

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing

dataset = read_csv(r'/content/drive/MyDrive/Input/mhealth_raw_data_refined.csv')
dataset = np.array(dataset)
classes=dataset[:,-1]
dataset=np.delete(dataset,-1,axis=1)
#classes = to_categorical(classes)
dataset= preprocessing.scale(dataset)

In [ ]:
classes

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
from collections import Counter
counter = Counter(classes)
print(counter)

Counter({0.0: 27648, 1.0: 27648, 2.0: 27648, 3.0: 27648, 5.0: 27648, 6.0: 27648, 7.0: 27648, 8.0: 27648, 9.0: 27648, 10.0: 27648, 11.0: 27648, 4.0: 27648})


In [ ]:
#Deleting

result = np.where(classes == 0)[0]
gt=np.delete(classes,result)
classes=gt
dataset=np.delete(dataset,result,axis=0)

In [ ]:
classes=classes-1

In [ ]:
classes

array([0., 0., 0., ..., 8., 8., 8.])

In [ ]:
counter = Counter(classes)
print(counter)

Counter({0.0: 27648, 1.0: 27648, 2.0: 27648, 3.0: 27648, 8.0: 27648, 6.0: 26676, 7.0: 26470, 5.0: 25857, 9.0: 24576, 10.0: 24576, 4.0: 24576, 11.0: 8243})


# For Oversampling 

In [ ]:

from imblearn.over_sampling import SMOTE
oversample = SMOTE()
dataset, classes = oversample.fit_resample(dataset, classes)
counter = Counter(classes)
print(counter)

Counter({0.0: 27648, 1.0: 27648, 2.0: 27648, 3.0: 27648, 5.0: 27648, 6.0: 27648, 7.0: 27648, 8.0: 27648, 9.0: 27648, 10.0: 27648, 11.0: 27648, 4.0: 27648})


In [ ]:
classes

array([ 0.,  0.,  0., ..., 11., 11., 11.])

In [ ]:
classes.shape

(98916,)

In [ ]:
from sklearn.preprocessing import OneHotEncoder 
onehotencoder = OneHotEncoder() 
y = onehotencoder.fit_transform(np.array(classes).reshape(-1,1)).toarray()
classes=y

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(dataset,classes, test_size=0.25, random_state=42)
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

      


(248832, 1, 12) (248832, 12) (82944, 1, 12) (82944, 12)


## LSTM

In [ ]:
loss='categorical_crossentropy'
number_of_classes=train_y.shape[1]
metrics=['accuracy']
model = Sequential()
model.add(LSTM(150, input_shape=(train_X.shape[1], train_X.shape[2]),dropout=0.10))
model.add(Dense(number_of_classes, activation='softmax'))
model.compile(loss=loss, optimizer='adam', metrics=metrics)
filepath = "/content/drive/MyDrive/Input/best-model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.summary()
history = model.fit(train_X, train_y, validation_split=0.25, epochs=50, verbose=1, callbacks=callbacks_list)


In [ ]:
# load best weights
model.load_weights("/content/drive/MyDrive/Input/best-model.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
Y_pred_test = model.predict(test_X)
y_pred_test = np.argmax(Y_pred_test, axis=1)

from sklearn.preprocessing import OneHotEncoder 
onehotencoder = OneHotEncoder() 
y_pred_test = onehotencoder.fit_transform(np.array(y_pred_test).reshape(-1,1)).toarray()

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print(classification_report(test_y,  y_pred_test))
from sklearn.metrics import confusion_matrix

#cm = confusion_matrix(test_y,  y_pred_test)
#print(cm)
accuracy_score(test_y, y_pred_test)*100

## GRU

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.layers import GRU

loss='categorical_crossentropy'
number_of_classes=train_y.shape[1]
metrics=['accuracy']
model = Sequential()
model.add(GRU(150, input_shape=(train_X.shape[1], train_X.shape[2]),dropout=0.10))
model.add(Dense(number_of_classes, activation='softmax'))
model.compile(loss=loss, optimizer='adam', metrics=metrics)
filepath = "/content/drive/MyDrive/Input/best-model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.summary()
history = model.fit(train_X, train_y, validation_split=0.25, epochs=50, verbose=1, callbacks=callbacks_list)


In [ ]:
# load best weights
model.load_weights("/content/drive/MyDrive/Input/best-model.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
Y_pred_test = model.predict(test_X)
y_pred_test = np.argmax(Y_pred_test, axis=1)

from sklearn.preprocessing import OneHotEncoder 
onehotencoder = OneHotEncoder() 
y_pred_test = onehotencoder.fit_transform(np.array(y_pred_test).reshape(-1,1)).toarray()

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print(classification_report(test_y,  y_pred_test))
from sklearn.metrics import confusion_matrix

#cm = confusion_matrix(test_y,  y_pred_test)
#print(cm)
accuracy_score(test_y, y_pred_test)*100